In [ ]:
import numpy as np
import market_behavior_utils as mbu
from datetime import datetime as DT
import matplotlib.pyplot as plt
import pandas_datareader as pdr
import scipy.stats as sc

# Week 1 - Return Types, Setup, and Data Foundations


In [ ]:
start_date = DT(year=2023,month=1,day=1)
end_date = DT(year=2024,month=12,day=31)
stocks = ['GOOGL','NVDA']
financial_data = mbu.grab_ticker(start_date,end_date,stocks)

In [ ]:
financial_data.plot(y=[f"CUM_SIMPLE_{stock}" for stock in stocks])

In [ ]:
financial_data[[f"SIMPLE_{stock}" for stock in stocks]].plot(kind='hist',bins=50)

# Week 2 - Distributions + Cumulative Returns + Intro to Bonds

In [ ]:
simple_data_nvda, x_vals_nvda, norm_vals_nvda = mbu.gaussian_overlay_data(financial_data,f"SIMPLE_{stocks[0]}")

In [ ]:
plt.hist(simple_data_nvda,bins=50,density=True)
plt.plot(x_vals_nvda,norm_vals_nvda)

In [ ]:
daily_bond_data = mbu.get_bond_data(start_date,end_date,"DGS3MO")

In [ ]:
mbu.static_sharpe_df(start_date,end_date,stocks)

# Week 3 - Regression, Volatility, Bond Valuation

In [ ]:
window = "30D"
stocks = ['SPY','GOOGL','NVDA']
bond_type = 'DGS3MO'
full_financial_data = mbu.rolling_statistics(start_date,end_date,stocks,window)


In [ ]:
full_financial_data[[f"{window}_SHARPE_{stock}"for stock in stocks]].plot()

In [ ]:
full_financial_data[[f"{window}_STD_{stock}"for stock in stocks]].plot()

In [ ]:
full_financial_data[[f"{window}_EXCESS_RETURN_{stock}" for stock in stocks]].corr()

In [ ]:
full_financial_data.plot(x=f'{window}_EXCESS_RETURN_{stocks[0]}',y=f'{window}_EXCESS_RETURN_{stocks[2]}',kind='scatter')

In [ ]:
daily_market_excess = (full_financial_data[f"SIMPLE_{stocks[0]}"] - full_financial_data[f"{bond_type}"]).dropna()
daily_stock_excess = (full_financial_data[f"SIMPLE_{stocks[2]}"] - full_financial_data[f"{bond_type}"]).dropna()

fitted_line =  sc.linregress(daily_market_excess,daily_stock_excess)

In [ ]:
plt.scatter(daily_market_excess,daily_stock_excess)
plt.plot(daily_market_excess,fitted_line.intercept + fitted_line.slope * daily_market_excess,color='red')

In [ ]:
face_value = 1000
coupon_value = 10
time_years = 5
coupons_per_year = 2
step = 150
rates = np.linspace(0.001,0.5,step)

In [ ]:
pvs_symbolic = mbu.present_value_fv(rates,face_value,coupon_value,time_years,coupons_per_year)

plt.plot(rates,pvs_symbolic)
plt.plot(rates,[face_value for _ in range(step)])
plt.plot([(coupons_per_year)*(coupon_value)/(face_value) for _ in range(step)],pvs_symbolic)

In [ ]:
dur_symbolic = mbu.duration_fv(rates,face_value,coupon_value,time_years,coupons_per_year)
convex_symbolic = mbu.convexity_fv(rates,face_value,coupon_value,time_years,coupons_per_year)

In [ ]:
plt.plot(rates,dur_symbolic)

In [ ]:
plt.plot(rates,convex_symbolic)

In [ ]:
start = DT(2024,1,1)
end = DT(2024,12,31)
gdp_data_call  = pdr.fred.FredReader(['GDPC1','UNRATE','CPIAUCSL','FEDFUNDS','PCE','DGS3MO'],start,end)

In [ ]:
gdp_data_call.read()